# 0: Import

In [81]:
import pandas as pd
import numpy as np
import re
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('stopwords')
nltk.download('punkt')
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\16178\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\16178\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# 1： Read in dataset

In [6]:
df = pd.read_csv('bostonfoods.csv')
df

,id,name,type,rating,image
0,1,seoul jangteo,"Korean, Chinese, Noodle",4.1,https://lh3.googleusercontent.com/p/AF1QipM5qO...
1,2,pho basil,"Vietnamese, Thai, Seafood",4.4,https://media-cdn.tripadvisor.com/media/photo-...
2,3,fuji at woc,"Japanese, Sushi Bars, Cocktail Bars",4.6,https://images.getbento.com/accounts/983a102b9...
3,4,myung dong,"Korean, Bars",4.0,https://lh3.googleusercontent.com/p/AF1QipPaHZ...
4,5,earls prudential,"American, Cocktail Bar, brunch",4.4,https://lh3.googleusercontent.com/p/AF1QipN_tF...
5,6,del friscos double eagle steakhouse,"steakhouses, seafood, american",4.5,https://s3-media0.fl.yelpcdn.com/bphoto/TjVxi3...
6,7,bartaco seaport,tacos,4.4,https://lh3.googleusercontent.com/p/AF1QipPZ3r...
7,8,pho linh,vietnamese,4.4,https://lh3.googleusercontent.com/p/AF1QipM_RY...
8,9,chaji creamery,"ice cream, frozen yogurt, coffee, tea",4.8,https://lh3.googleusercontent.com/p/AF1QipOYkL...
9,10,matcha cafe maiko,"bubble tea, ice cream, frozen yogurt",4.7,https://s3-media0.fl.yelpcdn.com/bphoto/qxOnyD...


In [94]:
new_df = df.copy()
new_df.head(5)

,id,name,type,rating,image
0,1,seoul jangteo,"Korean, Chinese, Noodle",4.1,https://lh3.googleusercontent.com/p/AF1QipM5qO...
1,2,pho basil,"Vietnamese, Thai, Seafood",4.4,https://media-cdn.tripadvisor.com/media/photo-...
2,3,fuji at woc,"Japanese, Sushi Bars, Cocktail Bars",4.6,https://images.getbento.com/accounts/983a102b9...
3,4,myung dong,"Korean, Bars",4.0,https://lh3.googleusercontent.com/p/AF1QipPaHZ...
4,5,earls prudential,"American, Cocktail Bar, brunch",4.4,https://lh3.googleusercontent.com/p/AF1QipN_tF...


In [75]:
def cleaning_text(s):
    """lowercase all letters
    remove stopwords, punctuations, etc before feed to models"""
    s = str(s)
    s = s.lower()
    s = re.sub(r'[^a-zA-Z]', ' ', s)
    return s

def df_type(df):
    """enter a df that you want to chekc the datatype of"""
    return df.dtypes

def cleanTextcol(column):
    """
    Given a column, we want to use the helper function cleaning_text,
    tokenize each word into list, then after cleaning, we join the text"""
    
    if column not in new_df:
        print('this column does not exist')
    else:
        new_df[column] = new_df[column].apply(cleaning_text)
        new_df[column] = new_df[column].apply(word_tokenize)
        new_df[column] = new_df[column].apply(lambda x: ' '.join(x))
        print('process completed')

In [104]:
cleanTextcol('type')

process completed


In [107]:
# save the cleaned version of dataset for later use in the web app
new_df.to_csv('cleaned_data.csv')

In [78]:
# instantiate TFIDF object, this is the algorithm we will use for recommendation system
tfidf = TfidfVectorizer()
TFmatrix = tfidf.fit_transform(new_df['type']) # transform to the type column
print(TFmatrix.shape)

(44, 55)


In [83]:
similarity_matrix = linear_kernel(TFmatrix,TFmatrix) # linear_kernel is faster than cosine_similarity


In [85]:
# Next we will need to create a series that maps the index of the matrix to restuarant names

mapping = pd.Series(new_df.index, index = new_df['name'])
mapping

name
seoul jangteo                           0
pho basil                               1
fuji at woc                             2
myung dong                              3
earls prudential                        4
del friscos double eagle steakhouse     5
bartaco seaport                         6
pho linh                                7
chaji creamery                          8
matcha cafe maiko                       9
cafe luna                              10
fuji at assembly                       11
noodle street                          12
gyu kaku japnese bbq                   13
tatte bakery                           14
futago udon                            15
cha kung fong                          16
big fu café                            17
monument restaurant tavern             18
happy lamb                             19
liu yi shou                            20
spring shabu shabu                     21
shake shack                            22
taiyaki nyc                  

In [108]:
# Next we will make the recommender function that recommend restaurants using cosine similarity

def recommend(name, count):
    restaurant_index = mapping[name] # get the index of input name restaurant

    # get similarity values with other restaurants
    # similarity score = list of index and similarity matrix
    similarity_score = list(enumerate(similarity_matrix[restaurant_index]))

    # sort in descend order
    similarity_score = sorted(similarity_score, key = lambda x: x[1], reverse=True)

    # get the scores of top restaurants based on input count
    similarity_score = similarity_score[1:count+1]

    # return restaurants name using the mapping series
    indices = [i[0] for i in similarity_score]

    return new_df[['name', 'rating', 'image']].iloc[indices]

In [112]:
df = recommend('pho basil', 2)
df

,name,rating,image
7,pho linh,4.4,https://lh3.googleusercontent.com/p/AF1QipM_RY...
12,noodle street,4.3,https://lh3.googleusercontent.com/p/AF1QipOFFP...


In [114]:
df['name'].iloc[0]

'pho linh'

In [ ]:
# save a list of restaurants in pickle
dataframe = pd.read_csv('cleaned_data.csv')
